In [1]:
import pandas as pd
import os
import sys
import scipy.stats
sys.path.append(os.path.abspath('/mnt/c/Users/ochapman/Documents/Mesirov/Software/oscutils'))
import get
import oscutils
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
#os.chdir('/mnt/c/Users/ochapman/Documents/Mesirov/medullo_ecDNA/2021-05-13_p53')
print(os.getcwd())

/mnt/c/Users/ochapman/Documents/Mesirov/medullo_ecDNA/2021-05-13_p53


In [2]:
s4 = get.medullo_p53()
s1 = get.medullo_patients()
s4

,Source,somatic_mut_available,somatic_tp53_mut,REVEL,CADD,3' UTR TP53,ClinVAR/VEP,germline_mut_available
Sample,,,,,,,,
MB018,Archer,True,False,0.0,0.0,0.0,0.0,True
MB036,Archer,True,False,0.0,0.0,0.0,0.0,True
MB037,Archer,True,False,0.0,0.0,0.0,0.0,True
MB088,Archer,True,False,0.0,0.0,0.0,0.0,True
MB091,Archer,True,False,0.0,0.0,0.0,0.0,True
MB095,Archer,True,False,0.0,0.0,0.0,0.0,True
MB099,Archer,False,False,NaN,NaN,NaN,NaN,False
MB102,Archer,True,False,0.0,0.0,0.0,0.0,True
MB104,Archer,True,False,0.0,0.0,0.0,0.0,True


In [3]:
def get_p53_table():
    s4 = get.medullo_p53()
    s1 = get.medullo_patients()
    s1 = s1[["ecDNA",'Subgroup']]
    s1['ecDNA'] = s1.ecDNA.fillna('no')
    df = s1.merge(s4,left_index=True,right_index=True)
    return df

def fisherexact(tbl):
    _,p=scipy.stats.fisher_exact(tbl)
    print("p =",min(p/2,1), '(Fisher exact test)')
    t = tbl.to_numpy().flatten()
    print("Precision =",round(t[3]/(t[3]+t[1]),3))
    print("Recall =",round(t[3]/(t[3]+t[2]),3))
    return tbl

def chisq(tbl):
    chi2,p,_,_ = scipy.stats.chi2_contingency(tbl)
    print("chi^2 =",round(chi2,3),", p =",p, '(Chi-squared test)')
    t = tbl.to_numpy().flatten()
    print("Precision =",round(t[3]/(t[3]+t[1]),3))
    print("Recall =",round(t[3]/(t[3]+t[2]),3))
    return tbl

In [4]:
def chisq_somatic_all():
    df = get_p53_table()
    df = df[df.somatic_mut_available]
    tbl = pd.crosstab(index=df.ecDNA,columns=df.somatic_tp53_mut)
    chisq(tbl)
    return tbl
chisq_somatic_all()

chi^2 = 12.417 , p = 0.0004254447687476671 (Chi-squared test)
Precision = 0.533
Recall = 0.121


somatic_tp53_mut,False,True
ecDNA,,
no,322,7
yes,58,8


In [5]:
def fisher_somatic_SHH():
    df = get_p53_table()
    df = df[df.somatic_mut_available]
    df = df[df.Subgroup == 'SHH']
    tbl = pd.crosstab(index=df.ecDNA,columns=df.somatic_tp53_mut)
    fisherexact(tbl)
    return tbl
fisher_somatic_SHH()

p = 2.481508029505076e-05 (Fisher exact test)
Precision = 0.889
Recall = 0.333


somatic_tp53_mut,False,True
ecDNA,,
no,69,1
yes,16,8


In [6]:
def chisq_germline_somatic_all():
    df = get_p53_table()
    df = df[df.somatic_mut_available & df.germline_mut_available]
    df['p53_mut'] = df.somatic_tp53_mut | df['ClinVAR/VEP'].astype(bool)
    tbl = pd.crosstab(index=df.ecDNA,columns=df.p53_mut)
    chisq(tbl)
    return tbl
chisq_germline_somatic_all()

chi^2 = 27.642 , p = 1.4598196396231796e-07 (Chi-squared test)
Precision = 0.591
Recall = 0.203


p53_mut,False,True
ecDNA,,
no,316,9
yes,51,13


In [7]:
def fisher_germline_somatic_SHH():
    df = get_p53_table()
    df = df[df.somatic_mut_available & df.germline_mut_available]
    df = df[df.Subgroup == 'SHH']
    df['p53_mut'] = df.somatic_tp53_mut | df['ClinVAR/VEP'].astype(bool)
    tbl = pd.crosstab(index=df.ecDNA,columns=df.p53_mut)
    fisherexact(tbl)
    return tbl
fisher_germline_somatic_SHH()

p = 1.290443022147928e-07 (Fisher exact test)
Precision = 0.857
Recall = 0.522


p53_mut,False,True
ecDNA,,
no,67,2
yes,11,12


In [19]:
def fisher_germline_somatic_sg(sg):
    df = get_p53_table()
    df = df[df.somatic_mut_available & df.germline_mut_available]
    df = df[df.Subgroup == sg]
    df['p53_mut'] = df.somatic_tp53_mut | df['ClinVAR/VEP'].astype(bool)
    tbl = pd.crosstab(index=df.ecDNA,columns=df.p53_mut)
    try:
        fisherexact(tbl)
    except Exception as error :
        print(error)
    return tbl
fisher_germline_somatic_sg('G4')

p = 0.1264879916463627 (Fisher exact test)
Precision = 0.5
Recall = 0.043


p53_mut,False,True
ecDNA,,
no,146,1
yes,22,1


In [20]:
fisher_germline_somatic_sg('G3') # no p53mut G3 samples.

The input `table` must be of shape (2, 2).


p53_mut,False
ecDNA,
no,79
yes,18
